In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time
from reconstruct_functions import *

In [2]:
hop_length = 2
win_length = 8 ###at least 4*hop_length, even number
channels = win_length//2+1

def load_wav_to_torch(full_path):
    sampling_rate, data = read(full_path)
    data = data.astype(np.float64)
    # data = np.round(data,decimals=6)
    return torch.DoubleTensor(data), sampling_rate


START =82698
audio_origin, sampling_rate = load_wav_to_torch('demo.wav')
audio_origin = audio_origin[None,START:START+20]

# audio_origin = torch.rand((1,wav_length), dtype=torch.float64)

print(audio_origin.shape)
print(audio_origin[0,:10])

torch.Size([1, 20])
tensor([-0.0663, -0.0522, -0.0448, -0.0367, -0.0226, -0.0133, -0.0003,  0.0126,
         0.0262,  0.0389], dtype=torch.float64)


In [3]:
# stft_fn = stft.STFT(filter_length=4, hop_length=1, win_length=4,
#                     window='hann')

stft_fn = stft.STFT(filter_length=win_length, hop_length=hop_length, win_length=win_length,
                    window=None)


def compare(a,b):
    return torch.mean(torch.abs(a-b)), torch.mean((a-b)*(a-b))

def compare_L1(ori,gen):
    return torch.mean(torch.abs(ori-gen)/torch.abs(ori))


def compare_L2(a,b):
    return torch.sum(torch.abs(a-b)), torch.sum((a-b)*(a-b))


In [4]:
magnitude, phase_origin = stft_fn.transform(audio_origin)

forward_basis = stft_fn.forward_basis
print(forward_basis.shape)
start_frame = 0
print(magnitude[0,:,start_frame])
magnitude    = magnitude[:,:,2:-2]
print(magnitude.shape)


torch.Size([10, 1, 8])
tensor([0.3561, 0.0656, 0.0008, 0.0218, 0.0008], dtype=torch.float64)
torch.Size([1, 5, 7])


In [5]:
ans = reconstruct_from_S_with_H_2(magnitude, stft_fn, hop_length=2, frame_number=None)


100%|██████████| 4/4 [00:00<00:00, 35.45it/s]


In [6]:
if ans[0,0]*audio_origin[0,0]<0:
    ans[:,:] = -ans[:,:]

In [7]:

start_frame = 0
end_frame = 1900
T_start = start_frame * hop_length + win_length-hop_length
T_end = end_frame * hop_length + win_length-hop_length
# print(ans[0,T_start:T_end])
# print(audio_origin[0,T_start:T_end])
print(torch.mean(torch.abs(audio_origin[0,T_start:T_end] - ans[0,T_start:T_end])))

tensor(3.5076e-13, dtype=torch.float64)


In [8]:

print(ans[0,-20:])
print(audio_origin[0,-20:])
print(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))

[-0.06625904 -0.05217025 -0.04480392 -0.03666257 -0.02256725 -0.01326675
 -0.00030922  0.01262541  0.0261763   0.03892441  0.04308636  0.04995391
  0.05208456  0.05477565  0.06344106  0.06448425  0.07074052  0.07725294
  0.07963908  0.07844818]
tensor([-0.0663, -0.0522, -0.0448, -0.0367, -0.0226, -0.0133, -0.0003,  0.0126,
         0.0262,  0.0389,  0.0431,  0.0500,  0.0521,  0.0548,  0.0634,  0.0645,
         0.0707,  0.0773,  0.0796,  0.0784], dtype=torch.float64)
tensor(4.9103e-13, dtype=torch.float64)


In [22]:
def reconstruct_from_S_with_H_2_block_test(magnitude, stft_fn, hop_length=2):
    #input shape: [1, channels, 3]
    
    channels = magnitude.shape[1]
    win_length = (channels-1)*2
    L = magnitude_to_L(magnitude, stft_fn.forward_basis[:2*hop_length+1,:,:])
    L = L.numpy()
    # print('R shape', L.shape)
    print('L sum',np.sum(np.abs(L)))
    print('magnitude sum', torch.sum(torch.abs(magnitude)))
    minimum_error = 1e10
    minimum_ans = np.zeros((1, win_length+2*hop_length)).astype(np.float64)

    for i in range(2**6):
        sgn10 = (i%(2**1))//(2**0)
        # sgn10 = 1 
        T10 = (sgn10 * 2 - 1) * np.sqrt(L[0,0] + 2*L[1,0] + 2*L[2,0] + 2*L[3,0] + 2*L[4,0])
        sgn20 = (i%(2**2))//(2**1)
        # sgn20 = 0
        T20 = (sgn20 * 2 - 1) * np.sqrt(L[0,0] - 2*L[1,0] + 2*L[2,0] - 2*L[3,0] + 2*L[4,0])
        S10 = (T10+T20)/2
        S20 = (T10-T20)/2
    
        sgn11 = (i%(2**3))//(2**2)
        # sgn11 = 1 
        T11 = (sgn11 * 2 - 1) * np.sqrt(L[0,1] + 2*L[1,1] + 2*L[2,1] + 2*L[3,1] + 2*L[4,1])
        sgn21 = (i%(2**4))//(2**3)
        # sgn21 = 0
        T21 = (sgn21 * 2 - 1) * np.sqrt(L[0,1] - 2*L[1,1] + 2*L[2,1] - 2*L[3,1] + 2*L[4,1])
        S11 = (T11+T21)/2
        S21 = (T11-T21)/2
    
        sgn12 = (i%(2**5))//(2**4)
        # sgn12 = 1 
        T12 = (sgn12 * 2 - 1) * np.sqrt(L[0,2] + 2*L[1,2] + 2*L[2,2] + 2*L[3,2] + 2*L[4,2])
        sgn22 = (i%(2**6))//(2**5)
        # sgn22 = 0
        T22 = (sgn22 * 2 - 1) * np.sqrt(L[0,2] - 2*L[1,2] + 2*L[2,2] - 2*L[3,2] + 2*L[4,2])
        S12 = (T12+T22)/2
        S22 = (T12-T22)/2
    
        x8_0 = S11 - S10
        x9_1 = S21 - S20
        x10_2 = S12 - S11
        x11_3 = S22 - S21
    
        A_matrix = np.zeros((10,8), dtype=np.double)
        b = np.zeros((10,1), dtype=np.double)
    
    
        ### equations for 0-4
        A_matrix[0,0] = 2*x8_0
        A_matrix[0,1] = 2*x9_1
        b[0,0] = L[0,1] - L[0,0] - x8_0**2 - x9_1**2
    
    
        A_matrix[1,1], A_matrix[1,7] = x8_0, x8_0
        A_matrix[1,0], A_matrix[1,2] = x9_1, x9_1
        b[1,0] = L[1,1] - L[1,0]-x9_1*x8_0
    
        A_matrix[2,2], A_matrix[2,6] = x8_0, x8_0
        A_matrix[2,3], A_matrix[2,7] = x9_1, x9_1
        b[2,0] = L[2,1] - L[2,0]
    
        A_matrix[3,3], A_matrix[3,5] = x8_0, x8_0
        A_matrix[3,4], A_matrix[3,6] = x9_1, x9_1
        b[3,0] = L[3,1] - L[3,0]
    
        A_matrix[4,4]= x8_0
        A_matrix[4,5]= x9_1
        b[4,0] = L[4,1] - L[4,0]
    
        ### equations for 5-9
        A_matrix[5,2] = 2*x10_2
        A_matrix[5,3] = 2*x11_3
        b[5,0] = L[0,2] - L[0,1] - x10_2**2 - x11_3**2
    
    
        A_matrix[6,1], A_matrix[6,3] = x10_2, x10_2
        A_matrix[6,2], A_matrix[6,4] = x11_3, x11_3
        b[6,0] = L[1,2] - L[1,1] - x10_2*x9_1 - x11_3*x10_2
    
        A_matrix[7,0], A_matrix[7,4] = x10_2, x10_2
        A_matrix[7,1], A_matrix[7,5] = x11_3, x11_3
        b[7,0] = L[2,2] - L[2,1] - x10_2*x8_0 - x11_3*x9_1
    
        A_matrix[8,7], A_matrix[8,5] = x10_2, x10_2
        A_matrix[8,0], A_matrix[8,6] = x11_3, x11_3
        b[8,0] = L[3,2] - L[3,1] - x11_3*x8_0
    
        A_matrix[9,6]= x10_2
        A_matrix[9,7]= x11_3
        b[9,0] = L[4,2] - L[4,1]
    
       
        Q,R = np.linalg.qr(A_matrix) # qr decomposition of A
        Qb = np.dot(Q.T,b) # computing Q^T*b (project b onto the range of A)
        ans = np.linalg.solve(R,Qb) # solving R*x = Q^T*b
        
#         temp = A_matrix.T @ A_matrix
#         ans = (np.linalg.inv(temp)) @ (A_matrix.T @ b)
        
#         if np.linalg.matrix_rank(temp)<win_length:
#             print('not full rank, may be error')
        
#         b_hat = A_matrix @ ans
    
        test_L   = np.zeros((channels, 3)).astype(np.float64)
        ans_full = np.zeros((1, win_length + 2*hop_length)).astype(np.float64)
        ans_full[0, :win_length] = ans[:win_length, 0]
        ans_full[0, win_length]   = ans[0,0]+x8_0
        ans_full[0, win_length+1] = ans[1,0]+x9_1
        ans_full[0, win_length+2] = ans[2,0]+x10_2
        ans_full[0, win_length+3] = ans[3,0]+x11_3
        
        for tt in range(3):
            for i in range(2*hop_length+1):
                test_L[i,tt] = np.sum(ans_full[0, tt*hop_length:tt*hop_length+win_length] * \
                                       np.roll(ans_full[0, tt*hop_length:tt*hop_length+win_length],i)
                                      )
        test_L[-1, :] = test_L[-1, :]/2
        # print('######')
        # print(np.sum(np.abs(test_L - L)))
        # print('test_L', test_L)
        # print('L',L)
        # print(minimum_ans)
        
        if np.sum(np.abs(test_L - L))<minimum_error:
            minimum_error = np.sum(np.abs(test_L - L))
            minimum_ans[0,:] = ans_full[0,:]
    return minimum_ans, minimum_error


print(audio_origin[0,:12])
ans_test,ans_error = reconstruct_from_S_with_H_2_block_test(magnitude[:,:,:3], stft_fn, hop_length=2)
print(ans_test)

L = magnitude_to_L(magnitude[:,:,:3], stft_fn.forward_basis[:2*hop_length+1,:,:])
L = L.numpy()

test_L   = np.zeros((channels, 3)).astype(np.float64)
        
for tt in range(3):
    for i in range(2*hop_length+1):
        test_L[i,tt] = np.sum(ans_test[0, tt*hop_length:tt*hop_length+win_length] * \
                                       np.roll(ans_test[0, tt*hop_length:tt*hop_length+win_length],i)
                                      )
test_L[-1, :] = test_L[-1, :]/2
print(L)
print(test_L)

tensor([-0.0663, -0.0522, -0.0448, -0.0367, -0.0226, -0.0133, -0.0003,  0.0126,
         0.0262,  0.0389,  0.0431,  0.0500], dtype=torch.float64)
L sum 0.0584165073426763
magnitude sum tensor(1.2228, dtype=torch.float64)
[[-0.06625904 -0.05217025 -0.04480392 -0.03666257 -0.02256725 -0.01326675
  -0.00030922  0.01262541  0.0261763   0.03892441  0.04308636  0.04995391]]
[[ 0.01130831  0.00639663  0.00739692]
 [ 0.00772723  0.00237501  0.0043511 ]
 [ 0.00558015 -0.00077955  0.00176002]
 [ 0.00425689 -0.00244227 -0.00025812]
 [ 0.00173839 -0.00155615 -0.00048976]]
[[ 0.01130831  0.00639663  0.00739692]
 [ 0.00772723  0.00237501  0.0043511 ]
 [ 0.00558015 -0.00077955  0.00176002]
 [ 0.00425689 -0.00244227 -0.00025812]
 [ 0.00173839 -0.00155615 -0.00048976]]
